<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Thresholds_tuning_trainer_11_120_ml1024_ep5_lr1e_5_th05_at1024_fp16_CLEANED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate
!pip install -q huggingface_hub
!pip install -q scikit-learn

# transformers and datasets are Hugging Face libraries
!pip install -q transformers datasets

!pip install -q wandb

In [ ]:
import datetime
import json
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time
import torch
import torch.nn.functional as F
import wandb

In [ ]:
from datasets              import DatasetDict
from google.colab          import auth, drive, files, userdata
from huggingface_hub       import create_repo, HfApi, login, upload_file, hf_hub_download, whoami
from huggingface_hub.utils import RepositoryNotFoundError
from sklearn.metrics       import accuracy_score, average_precision_score, classification_report, f1_score, precision_score, precision_recall_curve, precision_recall_fscore_support, recall_score, roc_auc_score
from torch.utils.data      import DataLoader
from tqdm.auto             import tqdm
from transformers          import AdamW, EvalPrediction, LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
from torch.nn              import BCEWithLogitsLoss, Module

# Hugging Face Hub (HF Hub) authenticate

In [ ]:
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")    # Store the key in os.environ
hf_token               = os.environ.get('HF_TOKEN')

login(token=hf_token)

# Check
user = whoami(token=hf_token)
assert user['name'] == 'claudelepere', f"{user['name']} is not claudelepere"
print(f"user: {user}")


#Create the model and dataset repos on HF Hub

In [ ]:
repo_id         = "claudelepere/skill_classification"                                             # repo_id = my namespace/username_or_org/reponame
model_repoUrl   = create_repo(repo_id=repo_id, repo_type="model",   private=True, exist_ok=True)
dataset_repoUrl = create_repo(repo_id=repo_id, repo_type="dataset", private=True, exist_ok=True)

print(f"Model Repo Url: {model_repoUrl} created successfully as a private repo")
print(f"Dataset Repo Url: {dataset_repoUrl} created successfully as a private repo")

In [ ]:
"""
# Check the Python version
print(sys.version)
print()

# Get the installed packages (you can see that conda is not installed (do not install it))
!pip list
print()

# Check system information
!cat /etc/os-release
!uname -m
print()

# Check the GPU details (only if the runtime type is T4 GPU)
#!nvidia-smi
#print()

# Check RAM
!free -h
print()

# Check disk space
!df -h
print()

# Get environment variables
for key, value in os.environ.items():
    print(f"{key}: {value}")
"""
!python -V

In [ ]:
print(f"currentdir: {os.getcwd()}")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

# Out Of Memory (OOM)

## OOM: reduce batch size
      small sizes (1 to 32):            PROs: better generalization in some cases
                                        CONs: may produce noisier gradients
      large sizes (128, 256, or higer): PROs: gradients are smoother, leading to more stable training
                                        CONs: poorer generalization (overfitting) in some cases
      intermediate sizes (32, 64):      combines the benefits of small and large sizes

In [ ]:
batch_size = 8

## OOM: enable gradient accumulation

* compensate for smaller batch sizes by accumulating gradients over several steps
* **effective batch size** = per-device batch size x gradient acumulation steps
* in each iteration, the model computes the gradients, these gradients are immediately used to update the model parameters

WARNING: gradient_accumulation_steps may not be None => comment it in TrainingArguments

In [ ]:
gradient_accumulation_steps = 4

## OOM: use PYTORCH_CUDA_ALLOC_CONF to handle memory fragmentation

In [ ]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## OOM: check for and kill zombie processes

In [ ]:
!ps aux | grep python
!kill -9 <PID>
if torch.cuda.is_available():
  !nvidia-smi
  print(torch.cuda.memory_summary())

## OOM: use fp16 (half precision) mixed precision training
reduces memory requirements by up to 50%

In [ ]:
fp16 = True

if fp16:
  _fp = "fp16"
else:
  _fp = "fp32"

## OOM: limit the number of GPU workers:
* 0 (default) or 1
* in Colab dataloader_num_workers = 1

## OOM: reduce model size or input tokens
* LongformerTokenizer.from_pretrained('allenai/longformer-base/large-4096'): large/base: 435M/149M parameters
* max_length: 4096 max for Longformer
* a single word can be equal to several tokens; stop words are **NOT discarded**!
* word_text_length_counts_sorted:
      jobs count                 : 50000
      jobs count under  512 words: 44794  89.59%
      jobs count under  640 words: 47894  95.79%
      jobs count under  768 words: 49123  98.25%
      jobs count under  896 words: 49691  99.38%
      jobs count under 1024 words: 49917  99.83%
      jobs count under 2048 words: 50000 100.00%
      jobs count under 4096 words: 50000 100.00%

In [ ]:
#max_length =  768    #      37 min    #
max_length = 1024    #      38 min    # GPU RAM: 12.2 / 40 GB
#max_length = 2048    # 1 hr 10 min    # GPU RAM: 21.4 / 40 GB
#max_length = 4096    # 2 hr 10 min    # GPU RAM: 39.5 / 40 GB => OutOfMemoryError

## OOM: free up GPU memory

In [ ]:
torch.cuda.empty_cache()

## OOM: reduce the number of transformers layers

In [ ]:
# hidden_layers = 6  # default:12

# Skills and datapoints

In [ ]:
skills         = 11
num_datapoints = 12000

datasetDict_zip_file_name = f"dataset_{skills}_{num_datapoints}.zip"
datasetDict_dir_name      = os.path.splitext(datasetDict_zip_file_name)[0]

print(f"datasetDict_zip_file_name: {datasetDict_zip_file_name}")
print(f"datasetDict_dir_name     : {datasetDict_dir_name}")
print()

# epoch
* 1 epoch is a complete pass through the entire training dataset
* with n datapoints and batch size = b, n/b iterations to complete 1 epoch
* 1 iteration is a single update of the model's parameters

In [ ]:
epochs = 5

# learning rate
* A common rule is to scale the learning rate proportionaly with the effective batch size
* **note: get_linear_schedule_with_warmup**

In [ ]:
learning_rate = 1e-5  # 1e-5 x 32/8

# threshold
default: 0.5

In [ ]:
threshold = 0.5

# attention window size

In [ ]:
attention_window = 1024 #512

# run_name

In [ ]:
run_name = f"{skills}_{num_datapoints}_ml{max_length}_ep{epochs}_lr{learning_rate}_th{threshold}_at{attention_window}_{_fp}"

if 'gradient_accumulation_steps' not in globals():
  run_name = f"{run_name}_ba{batch_size}"
else:
  run_name = f"{run_name}_ba{batch_size}x{gradient_accumulation_steps}"

print(f"run_name: {run_name}")

# Upload and unzip job dataset

In [ ]:
def upload_unzip_dataset(file_name=datasetDict_zip_file_name):
  # Check if the file exists
  if not os.path.exists(file_name):
    print(f"'{file_name}' not found in /content. Uploading...")
    uploaded_files = files.upload()                              # Prompt file upload dialog
    if file_name not in uploaded_files:
      raise FileNotFoundError(f"'{file_name}' was not uploaded. Please try again.")
    print(f"'{file_name}' successfully uploaded to /content")
    uploaded_file_name = list(uploaded_files.keys())[0]          # Get the name of the uploaded file

    !unzip {uploaded_file_name}

    unzipped_dir_name = os.path.splitext(uploaded_file_name)[0]
    assert unzipped_dir_name==datasetDict_dir_name, "unzipped_dir_name != datasetDict_dir_name"
  else:
    print(f"'{datasetDict_dir_name}' already exists in /content.")

In [ ]:
upload_unzip_dataset(datasetDict_zip_file_name)

# W&B initialization

In [ ]:
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")        # Store the key in os.environ
wandb_api_key               = os.environ.get('WANDB_API_KEY')
wandb.login(key=wandb_api_key)

In [ ]:
try:
  wandb.init(
      project = "skill_classification",
      name    = run_name,
      entity  = "claudelepere-c-cile-cy",
      config  = {
          "learning_rate": learning_rate,
          "epochs"       : epochs,
          "batch_size"   : batch_size
      }
  )
except wandb.errors.CommError as err:
  print(f"CommError: {err}")
except Exception as exc:
  print(f"Exception: {exc}")

# Create datasetDict (HF DatasetDict) = 3 HF Dataset, train, validation and test

In [ ]:
datasetDict = DatasetDict.load_from_disk(datasetDict_dir_name)

In [ ]:
print(f"datasetDict: {type(datasetDict)} {datasetDict.shape}\n{datasetDict}")
print(f"datasetDict.keys(): {datasetDict.keys()}")
print(f"datasetDict['train']:      {type(datasetDict['train'])}      {datasetDict['train'].shape}")
print(f"datasetDict['validation']: {type(datasetDict['validation'])} {datasetDict['validation'].shape}")
print(f"datasetDict['test']:       {type(datasetDict['test'])}       {datasetDict['test'].shape}")

In [ ]:
example = datasetDict['train'][0]
print(f"datasetDict['train'][0]: {type(example)} {example.keys()}\n{example}")

# Create labels (list), id2label (dict) and label2id (dict).
* dataset 7_1000_125_125  ,  48 labels
* dataset 7_128_18_54     ,  42 labels
* dataset 8910_1087_68_204, 206 labels
* dataset 11_1000         ,   6 labels

In [ ]:
labels = [label for label in datasetDict['train'].features.keys() if label not in ['id', 'text']]
labels.sort()
print(f"labels: {type(labels)} {len(labels)}\n{labels}")
num_labels = len(labels)

In [ ]:
id2label = {idx: label for idx, label in enumerate(labels)}
print(f"id2label: {type(id2label)} {len(id2label)}\n{id2label}")

In [ ]:
label2id = {label: idx for idx, label in enumerate(labels)}
print(f"label2id: {type(label2id)} {len(label2id)}\n{label2id}")

# Load the tokenizer and the model

In [ ]:
model_name = "allenai/longformer-base-4096"

In [ ]:
tokenizer = LongformerTokenizerFast.from_pretrained(model_name)

In [ ]:
model = LongformerForSequenceClassification.from_pretrained(
    model_name,
    num_labels   = num_labels,
    id2label     = id2label,
    label2id     = label2id,
    problem_type = 'multi_label_classification'
)

# Configure attention window size
model.config.attention_window = attention_window

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

## Tokenize ('input_ids' and 'attention_mask'), add 'global_attention_mask' (for Longformer), add 'labels'

In [ ]:
def preprocess_data(examples, indices):
  # Step 1: Extract text and tokenize
  text = examples['text']             # Batch of texts
  encoding = tokenizer(
      text,                           # Tokenize text
      truncation     = True,
      padding        = 'max_length',
      max_length     = max_length,
      return_tensors = 'pt'           # Return PyTorch tensors
  )

  # Step 2: Create and add the global attention mask
  global_attention_mask             = torch.zeros_like(encoding['input_ids'])  # Initialize global attention mask with zeros (same shape as input_ids)
  global_attention_mask[:, 0]       = 1                                        # Set global attention on the first token ([CLS], token ID=0) in each sequence
  encoding['global_attention_mask'] = global_attention_mask                    # Add the global_attention_mask to the batch

  # Step 3: Create and populate the label matrix
  labels_matrix = torch.zeros((len(text), len(labels)), dtype=torch.float32)   # Create an empty label matrix
  #print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")
  #---------Populate label matrix
  for idx, label in enumerate(labels):
    #print(f"idx:{idx} label:{label}")
    if label in examples:
      labels_matrix[:, idx] = torch.tensor(
          [1.0 if val else 0.0 for val in examples[label]],
          dtype=torch.float32
          )
  print(f"labels_matrix: {type(labels_matrix)} {labels_matrix.shape}")

  encoding['labels'] = labels_matrix                                           # Add labels to the encoding
  print(f"encoding['labels']: {type(encoding['labels'])} {encoding['labels'].shape}")

  # encoding: <class 'transformers.tokenization_utils_base.BatchEncoding'> dict_keys(['input_ids', 'attention_mask', 'global_attention_mask', 'labels'])
  #   'input_ids': tensor([[
  #   'attention_mask': tensor([[
  #   'global_attention_mask': tensor([[
  #   'labels': tensor([[
  #print(f"1 preprocess_data call: encoding: {type(encoding)} {encoding.keys()}")

  return encoding

# Create encoded_dataset (datasets.dataset_dict.DatasetDict) = 3 encoded datasets.arrow_dataset.Dataset, train, validation and test

In [ ]:
encoded_dataset = datasetDict.map(
    preprocess_data,
    batched        = True,
    remove_columns = datasetDict['train'].column_names,
    with_indices   = True
)

print(f"Zencoded_dataset: {type(encoded_dataset)} shape={encoded_dataset.shape}")

In [ ]:
encoded_dataset.set_format('torch')
train_dataset      = encoded_dataset['train']
validation_dataset = encoded_dataset['validation']
test_dataset       = encoded_dataset['test']

print(f"train_dataset_tensor:                          {type(train_dataset)}                              {train_dataset.shape} {train_dataset.features}\n{train_dataset}")
print(f"train_dataset_tensor['input_ids']:             {type(train_dataset['input_ids'])}             len={len(train_dataset['input_ids'])}             shape={train_dataset['input_ids'].shape}            ") #\n{train_dataset['input_ids']}")
print(f"train_dataset_tensor['attention_mask']:        {type(train_dataset['attention_mask'])}        len={len(train_dataset['attention_mask'])}        shape={train_dataset['attention_mask'].shape}       ") #\n{train_dataset['attention_mask']}")
print(f"train_dataset_tensor['global_attention_mask']: {type(train_dataset['global_attention_mask'])} len={len(train_dataset['global_attention_mask'])} shape={train_dataset['global_attention_mask'].shape}") #\n{train_dataset['global_attention_mask']}")
print(f"train_dataset_tensor['labels']:                {type(train_dataset['labels'])}                len={len(train_dataset['labels'])}                shape={train_dataset['labels'].shape}               ") #\n{train_dataset['labels']}")

# Truncated part

In [ ]:
def get_truncated_part(text):
  tokens = tokenizer(
      text,
      truncation                = True,
      padding                   = 'max_length',
      max_length                = max_length,
      return_overflowing_tokens = True,
      return_tensors            = None
      )
  print(f"tokens.keys(): {tokens.keys()}")

  # Get the truncated tokens
  truncated_ids = tokens["input_ids"][0]
  print(f"truncated_ids: {type(truncated_ids)} {truncated_ids}")
  #overflow_ids  = tokens["overflow_to_sample_mapping"][0]
  #print(f"overflow_ids: {type(overflow_ids)} {overflow_ids}")

  # Decode the tokens back to text
  truncated_text = tokenizer.decode(truncated_ids, skip_special_tokens=True)
  #overflow_text  = tokenizer.decode(overflow_ids, skip_special_tokens=True)

  print(f"original_text :\n{text}")
  print(f"truncated_text:\n{truncated_text}")
  #print(f"overflow_text:\n{overflow_text}")

  original_tokens  = tokenizer.tokenize(text)
  truncated_tokens = tokenizer.tokenize(truncated_text)
  #overflow_tokens  = tokenizer.tokenize(overflow_text)

  print(f"original_tokens count : {len(original_tokens)}")
  print(f"truncated_tokens count: {len(truncated_tokens)}")
  #print(f"overflow_tokens count: {len(overflow_tokens)}")

In [ ]:
example_text = datasetDict['train'][0]['text']
#get_truncated_part(example_text)

In [ ]:
inputs = tokenizer(
    example_text,
    truncation     = True,
    padding        = 'max_length',
    max_length     = max_length,
    return_tensors = 'pt'
)

# Forward pass for multi-label classification

In [ ]:
outputs = model(
    input_ids      = inputs.input_ids,
    attention_mask = inputs.attention_mask
    )

In [ ]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

In [ ]:
# Logits (= raw model outputs)
logits = outputs.logits
print(f"logits: {type(logits)} {logits.shape}\n{logits}")

In [ ]:
# Convert logits to probabilities
sigmoid = torch.nn.Sigmoid()
probs   = sigmoid(logits)
print(f"probs: {type(probs)} {probs.shape}\n{probs}")

In [ ]:
example = encoded_dataset['train'][0]

In [ ]:
print(f"example: {type(example)} {example.keys()}\n{example}")
print()
#print(f"example['input_ids']: {type(example['input_ids'])} {len(example['input_ids'])}\n{example['input_ids']}")
#print(f"example['attention_mask']: {type(example['attention_mask'])} {len(example['attention_mask'])}\n{example['attention_mask']}")
#print(f"example['labels']:  {type(example['labels'])} {len(example['labels'])}\n{example['labels']}")

In [ ]:
tokenizer.decode(example['input_ids'])

In [ ]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

# Set PyTorch format to ensures correctness and compatibility with PyTorch pipelines
The 3 Hugging Face Dataset are formatted as PyTorch Dataset.

In [ ]:
encoded_dataset.set_format('torch')

# Workflow

# - 3 steps: training, evaluation, prediction/inference

# - 3 datasets: train, validation, test

# - 3 Trainer functions: train, evaluate, predict


# A unique commit_message for all uploads to HF Hub

In [ ]:
commit_message = f"{datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

print(f"commit_message: {commit_message}")

# Training step


In [ ]:
batch_size  = batch_size
metric_name = "f1"

## Metrics
  source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/

In [ ]:
def multi_label_metrics(logits, true_labels):
  """
  logits => sigmoid => probabilities => predictions

  Args:
    logits     : raw, unnormalized scores outputted by the model  (numpy array of shape (batch_size, num_labels))
    true_labels: actual labels for the data                       (numpy array of shape (batch_size, num_labels))

  Returns:
    metrics: dictionary of scores
  """
  average = 'micro'    # 'micro' or 'weighted'

  sigmoid = torch.nn.Sigmoid()
  probs   = sigmoid(torch.Tensor(logits))
  # next, use threshold to turn them into integer predictions
  preds = np.zeros(probs.shape)
  preds[np.where(probs > threshold)] = 1

  # compute metrics
  f1                   = f1_score               (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  precision            = precision_score        (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  recall               = recall_score           (y_true=true_labels, y_pred=preds,  average=average)    #, zero_division=1)
  roc_auc              = roc_auc_score          (y_true=true_labels, y_score=probs, average=average)
  precision_recall_auc = average_precision_score(y_true=true_labels, y_score=probs, average=average)
  accuracy             = accuracy_score         (y_true=true_labels, y_pred=preds)

  metrics = {

      'f1'                  : f1,
      'precision'           : precision,
      'recall'              : recall,
      'roc_auc'             : roc_auc,
      'precision_recall_auc': precision_recall_auc,
      'accuracy'            : accuracy
  }

  return metrics

In [ ]:
def compute_metrics(p: EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
  result = multi_label_metrics(logits=preds,true_labels=p.label_ids)   # true_labels=p.label_ids<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

  return result

In [ ]:
print(f"input_ids:              {type(encoded_dataset['train']['input_ids'][0])}\t{encoded_dataset['train']['input_ids'][0].shape}")
print(f"attention_mask:         {type(encoded_dataset['train']['attention_mask'][0])}\t{encoded_dataset['train']['attention_mask'][0].shape}")
print(f"global_attention_mask:  {type(encoded_dataset['train']['global_attention_mask'][0])}\t{encoded_dataset['train']['global_attention_mask'][0].shape}")
print(f"labels:                 {type(encoded_dataset['train'][0]['labels'])}\t{encoded_dataset['train'][0]['labels'].shape}")

## Execute a forward pass for debugging or verification purposes (cf. BERT_3_1 in Notion BERT database)

In [ ]:
outputs = model(
    input_ids      = encoded_dataset['train']['input_ids'][0].unsqueeze(0),
    attention_mask = encoded_dataset['train']['attention_mask'][0].unsqueeze(0),
    labels         = encoded_dataset['train'][0]['labels'].unsqueeze(0)
    )

In [ ]:
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

## Weighted loss function

In [ ]:
"""# Define the weighted loss function

class_weights = torch.tensor([7.68, 2.15, 0.61, 0.47, 0.68, 6.26], dtype=torch.float32).to(device)
loss_fn       = BCEWithLogitsLoss(pos_weight=class_weights)  # For multi-label classification (binary classification per label)

## Class supports, class weigths, weighted loss function

Reminder:
*   df_jobs      : <class 'pandas.core.frame.DataFrame'>
*   df_jobs['id']: <class 'pandas.core.series.Series'>

dataset = Dataset.from_pandas(df_jobs)
*   dataset      : <class 'datasets.arrow_dataset.Dataset'>
*   dataset['id']: <class 'list'>

*   dataset_dict_jobs : <class 'datasets.dataset_dict.DatasetDict'>
*   train_dataset     : <class 'datasets.arrow_dataset.Dataset'>
*   validation_dataset: <class 'datasets.arrow_dataset.Dataset'>
*   test_dataset      : <class 'datasets.arrow_dataset.Dataset'>


We calculate the class supports for the train, validation and test datasets; the class weights and the weighted loss function are used for training only; the class supports of validation_dataset and test_dataset are calculated for information only.

def get_train_class_weights(datasetDict, labels):
  print(f"datasetDict: {type(datasetDict)} shape={datasetDict.shape}\n{datasetDict}")
  print(f"labels: {type(labels)} len={len(labels)}\n{labels}")

  dataset_train      = datasetDict['train']
  dataset_validation = datasetDict['validation']
  dataset_test       = datasetDict['test']

  def calculate_class_supports(dataset, labels):
    class_supports = dataset.map(
        lambda example: {col: example[col] for col in labels},
        batched=True
    ).to_pandas()[labels].sum(axis=0)
    return class_supports

  class_supports = {}

  for split_name, split_dataset in datasetDict.items():
    class_supports[split_name] = calculate_class_supports(split_dataset, labels)

  for split_name, split_class_supports in class_supports.items():
    print(f"{split_name}: {type(split_class_supports)} len={len(split_class_supports)}\n{split_class_supports}")

  train_class_supports_list = class_supports['train'].tolist()
  print(f"train_class_supports_list: {type(train_class_supports_list)} len={len(train_class_supports_list)} {train_class_supports_list}")

  train_class_supports_tensor = torch.tensor(train_class_supports_list, dtype=torch.float32)
  print(f"train_class_supports_tensor: {type(train_class_supports_tensor)} len={len(train_class_supports_tensor)} {train_class_supports_tensor}")

  train_total_samples = dataset_train.num_rows
  print(f"train_total_samples: {train_total_samples}")

  number_of_classes = len(labels)
  print(f"number_of_classes: {number_of_classes}")

  train_class_weights = train_total_samples / (number_of_classes * train_class_supports_tensor)
  print(f"train_class_weights: {type(train_class_weights)} len={len(train_class_weights)} {train_class_weights}")

  train_class_weights_sum = train_class_weights.sum()
  print(f"train_class_weights_sum: {train_class_weights_sum}")

  normalized_train_class_weights = (train_class_weights / train_class_weights_sum) * number_of_classes
  print(f"normalized_train_class_weights: {type(normalized_train_class_weights)} len={len(normalized_train_class_weights)} {normalized_train_class_weights}")

  # Positives samples per label
  supports = train_class_supports_tensor
  print(f"supports: {type(supports)} {len(supports)} {supports}")

  # Negatives samples per label
  negatives = train_total_samples - supports
  print(f"negatives: {type(negatives)} {len(negatives)} {negatives}")

  # pos_weights = negative to positive ratios
  pos_weights = negatives/supports
  print(f"pos_weights: {type(pos_weights)} {len(pos_weights)} {pos_weights}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using min-max scaling
  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  # Normalize using z-score standardization
  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  # Normalize using sum-to-one
  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  #return normalized_pos_weights_sum1

pos_weights = get_train_class_weights(datasetDict, labels)

loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))  # For multi-label classification (binary classification per label)
print(f"loss_fn: {type(loss_fn)} {loss_fn}")
"""

In [ ]:
def get_class_weights(labels=encoded_dataset['train']['labels']):
  print(f"labels: {type(labels)} len={len(labels)} shape={labels.shape}\n{labels}")

  num_samples, num_labels = labels.shape
  print(f"num_samples: {type(num_samples)} {num_samples}")
  print(f"num_labels:  {type(num_labels)}  {num_labels}")

  class_counts = labels.sum(dim=0)
  print(f"class_counts: {type(class_counts)} len={len(class_counts)}\n{class_counts}")

  pos_weights = (num_samples-class_counts) / class_counts
  print(f"pos_weights: {type(pos_weights)} len={len(pos_weights)}\n{pos_weights}")

  normalized_pos_weights_minmax = (pos_weights - pos_weights.min()) / (pos_weights.max() - pos_weights.min())
  print(f"normalized_pos_weights_minmax: {type(normalized_pos_weights_minmax)} {len(normalized_pos_weights_minmax)} {normalized_pos_weights_minmax}")

  normalized_pos_weights_zscore = (pos_weights - pos_weights.mean()) / pos_weights.std()
  print(f"normalized_pos_weights_zscore: {type(normalized_pos_weights_zscore)} {len(normalized_pos_weights_zscore)} {normalized_pos_weights_zscore}")

  normalized_pos_weights_sum1 = pos_weights / pos_weights.sum()
  print(f"normalized_pos_weights_sum1: {type(normalized_pos_weights_sum1)} {len(normalized_pos_weights_sum1)} {normalized_pos_weights_sum1}")

  #return pos_weights
  #return normalized_pos_weights_minmax
  #return normalized_pos_weights_zscore
  return normalized_pos_weights_sum1

In [ ]:
pos_weights = get_class_weights()

In [ ]:
loss_fn = BCEWithLogitsLoss(pos_weight=pos_weights.to(device))

## FocalLoss

In [ ]:
class FocalLoss(Module):
  """
  Focal Loss implementation
  """
  def __init__(self, alpha=1.0, gamma=2.0, logits=False, reduce=True):
    super(FocalLoss, self).__init__()
    self.alpha   = alpha
    self.gamma   = gamma
    self.logits  = logits  # This flag is to indicate whether input is logits or probability
    self.reduce  = reduce

  # inputs  = model's predictions: PyTorch tensor, shape=(batch_size, num_classes)
  # targets = ground truth labels: PyTorch tensor, shape=same as inputs shape
  def forward(self, inputs, targets):
    # Here, we check if input is probability or logits
    if self.logits:
      # Input is logits
      BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
    else:
      # Input is probability
      BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
    pt = torch.exp(-BCE_loss)
    F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

    if self.reduce:
      return torch.mean(F_loss)
    else:
      return F_loss

  def __repr__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __str__(self):
    return f"FocalLoss(alpha={self.alpha}, gamma={self.gamma}, logits={self.logits}, reduce={self.reduce})"

  def __call__(self, inputs, targets):
    return self.forward(inputs, targets)

In [ ]:
focal_loss_fn = FocalLoss(alpha=0.5, gamma=4.0, logits=True, reduce=True)
print(f"focal_loss_fn: {type(focal_loss_fn)} {focal_loss_fn}")

## HF transformer Trainer and CustomTrainer
Abstracts the training loop.

In [ ]:
training_args = TrainingArguments(
    output_dir                  = './training_results',  # where model predictions and checkpoints will be written during training
    overwrite_output_dir        = True,
    logging_dir                 = './logs',
    logging_steps               = 50,
    save_steps                  = 500,
    save_total_limit            = 2,
    eval_strategy               = 'epoch',
    save_strategy               = 'epoch',
    learning_rate               = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size  = batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    num_train_epochs            = epochs,
    weight_decay                = 0.01,
    load_best_model_at_end      = True,
    metric_for_best_model       = metric_name,
    run_name                    = run_name,
    fp16                        = fp16,
    report_to                  = 'wandb'
    )

In [ ]:
class CustomTrainer(Trainer):

  def __init__(self, *args, loss_fn=None, **kwargs):
    super().__init__(*args, **kwargs)
    self.loss_fn = loss_fn

  """
  # No print in compute_loss because out of memory because prints are batch per batch
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):

    #print(f"inputs passed to compute_loss: {inputs.keys()}")
    #input_ids             = inputs['input_ids']                        # shape: batch_size, sequence_length
    #attention_mask        = inputs['attention_mask']                   # shape: batch_size, sequence_length
    #global_attention_mask = inputs.get('global_attention_mask', None)  # shape: batch_size, sequence_length; optional as LongFormer specific
    labels                = inputs.pop('labels', None)                 # shape: batch_size, num_labels; needed for loss computation, not required by the model

    #outputs = model(**inputs, global_attention_mask=global_attention_mask)  # Forward pass
    # Forward pass
    #outputs = model(
    #    input_ids             = input_ids,
    #    attention_mask        = attention_mask,
    #    global_attention_mask = global_attention_mask,
    #    labels                = labels
    #)
    outputs = model(**inputs, labels=labels)
    #print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")
    logits = outputs.logits  # shape: (batch_size, num_labels)

    # If labels are provided, compute loss
    if labels is not None:
      # Use the custom loss function if provided
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)  # Compute weighted loss
      else:
        # Default loss: BCEWithLogitsLoss
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)    # Compute loss
      return (loss, outputs) if return_outputs else loss

    # If no labels, return outputs only, for evaluation or inference
    return outputs
    """
  def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
    labels  = inputs.pop('labels', None)
    outputs = model(**inputs, labels=labels)
    logits  = outputs.logits

    if labels is not None:
      if self.loss_fn is not None:
        loss = self.loss_fn(logits, labels)
      else:
        loss_fn = BCEWithLogitsLoss()
        loss    = loss_fn(logits, labels)
      return (loss, outputs) if return_outputs else loss

    return outputs

In [ ]:
"""
trainer = CustomTrainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,                # custom metrics function
    loss_fn         = focal_loss_fn,
)
"""

trainer = Trainer(
    model           = model,
    args            = training_args,
    train_dataset   = encoded_dataset["train"],
    eval_dataset    = encoded_dataset["validation"],
    compute_metrics = compute_metrics,                # custom metrics function
)

In [ ]:
"""
predictions_output: <class 'transformers.trainer_utils.PredictionOutput'> len=3:
- predictions: np.ndarray          raw model outputs = logits
- label_ids  : np.ndarray or None: true labels corresponding to the predictions, if available
- metrics    : dict              : metrics computed during prediction step
"""
"""
predictions_output = trainer.predict(test_dataset)
print(f"predictions_output: {type(predictions_output)} len={len(predictions_output)}\n{predictions_output}")  # <class 'transformers.trainer_utils.PredictionOutput'> len=3
logits = predictions_output.predictions
print(f"logits: {type(logits)} len={len(logits)} shape={logits.shape}\n{logits}")


raise Exception("I'm here")
"""

## trainer.train

In [ ]:
trainer_train_results = trainer.train()

print(f"trainer_train_results: {type(trainer_train_results)} len={len(trainer_train_results)}\n{trainer_train_results}")
print()
print(f"trainer_train_results.metrics: {type(trainer_train_results.metrics)} len={len(trainer_train_results.metrics)}\n{json.dumps(trainer_train_results.metrics, indent=4)}")

In [ ]:
print("trainer.train successfully completed")

## trainer.train results: save locally and upload to HF Hub

In [ ]:
trainer_train_results_path = "trainer_train_results.json"
with open(trainer_train_results_path, "w") as f:
  json.dump(trainer_train_results, f)

print(f"trainer_train_results successfully saved locally to {trainer_train_results_path}")

In [ ]:
upload_file(
    path_or_fileobj = trainer_train_results_path,
    path_in_repo    = trainer_train_results_path,
    repo_id         = repo_id,
    repo_type       = 'dataset',
    commit_message  = commit_message
)

print(f"trainer_train_results successfully uploaded to HF Hub {repo_id} as {trainer_train_results_path}")

## trainer.train results: check that the uploaded file can be downloaded
File locally downloaded to:

/root/.cache/huggingface/hub/datasets-claudelepere-skill_classification/snapshots/full_commit_hash/trainer_train_results.json

In [ ]:
file_path = hf_hub_download(repo_type="dataset", repo_id=repo_id, filename=trainer_train_results_path)

print(f"file_path: {file_path}")

# Evaluation step

## Evaluation: trainer.evaluate
trainer.evaluate uses a fixed threshold of 0.5 to convert logits into binary labels, which is often suboptimal for imbalanced data.

In [ ]:
evaluation_trainer_evaluate_results = trainer.evaluate(
    #eval_dataset=encoded_dataset["validation"],  # by default, trainer.evaluate() evaluates the dataset passed as eval_dataset during training
    metric_key_prefix="eval"                     # prefix for the evaluation metrics
)

print(f"evaluation_trainer_evaluate_results: {type(evaluation_trainer_evaluate_results)} len={len(evaluation_trainer_evaluate_results)}\n{json.dumps(evaluation_trainer_evaluate_results, indent=4)}")


In [ ]:
print("evaluation_trainer.evaluate successfully completed")

## Evaluation: trainer.evaluate results: save locally and upload to HF Hub

In [ ]:
evaluation_trainer_evaluate_results_path = "evaluation_trainer_evaluate_results.json"
with open(evaluation_trainer_evaluate_results_path, "w") as f:
  json.dump(evaluation_trainer_evaluate_results, f)

print(f"evaluation_trainer_evaluate_results successfully saved locally to {evaluation_trainer_evaluate_results_path}")

In [ ]:
upload_file(
    path_or_fileobj = evaluation_trainer_evaluate_results_path,
    path_in_repo    = evaluation_trainer_evaluate_results_path,
    repo_id         = repo_id,
    repo_type       = 'dataset',
    commit_message  = commit_message
)

print(f"evaluation_trainer_evaluate_results successfully uploaded to HF Hub {repo_id} as {evaluation_trainer_evaluate_results_path}")

## Evaluation: trainer.predict

In [ ]:
def tune_thresholds(true_labels, probs, id2label):
  """
  Tune thresholds for each label to maximize F1 alone, as F1 balances precision and recall into a single metric.

  Args:
    true_labels: actual labels for the data                                      (numpy array of shape (num_samples, num_labels))
    probs      : predicted probabilities                                         (numpy array of shape (num_samples, num_labels))
    id2label   : dictionary mapping label indices (int) to label names (string)

  Returns:
    best_thresholds: best threshold for each label                                                      (numpy array of shape (num_labels,))
    best_metrics   : dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label (dictionary of numpy arrays)
  """
  thresholds      = np.linspace(0.1, 0.9, 9)
  best_thresholds = np.zeros(len(id2label))
  best_metrics    = {label: {'f1': 0.0, 'precision': 0.0, 'recall': 0.0} for label in id2label.values()}

  for label_idx, label in id2label.items():
    for threshold in thresholds:
      pred                     = (probs[:, label_idx] > threshold).astype(int)
      precision, recall, f1, _ = precision_recall_fscore_support(true_labels[:, label_idx], pred, average='binary', zero_division=0)
      if f1 > best_metrics[label]['f1']:
        best_thresholds[label_idx]       = threshold
        best_metrics[label]['f1']        = f1
        best_metrics[label]['precision'] = precision
        best_metrics[label]['recall']    = recall

  return best_thresholds, best_metrics


In [ ]:
def compute_metrics_with_threshold(probs, label_ids, thresholds, id2label):
  """
  Compute metrics during evaluation or test, by applying tuned thresholds
  """
  #logits  = eval_preds.predictions
  #labels  = eval_preds.label_ids
  #sigmoid = torch.nn.Sigmoid  # Sigmoid or numpy?
  #probs   = sigmoid(logits).cpu().numpy()
  preds   = np.zeros_like(probs)

  # Apply threshold per label
  for label_idx in id2label.keys():
    preds[:, label_idx] = (probs[:, label_idx] > thresholds[label_idx]).astype(int)

  # Compute metrics
  f1                    = f1_score               (label_ids, preds, average='micro')
  precision             = precision_score        (label_ids, preds, average='micro')
  recall                = recall_score           (label_ids, preds, average='micro')
  accuracy              = accuracy_score         (label_ids, preds)
  roc_auc               = roc_auc_score          (label_ids, probs, average='micro')
  precision_recall_auc  = average_precision_score(label_ids, probs, average='micro')

  # Use id2label for target_names
  report = classification_report(label_ids, preds, target_names=id2label.values(), zero_division=0)

  metrics = {
      'f1'                   : f1,
      'precision'            : precision,
      'recall'               : recall,
      'accuracy'             : accuracy,
      'roc_auc'              : roc_auc,
      'precision_recall_auc' : precision_recall_auc,
      'thresholds'           : thresholds.tolist(),
      'classification_report': report
  }

  return metrics


In [ ]:
def predict_with_thresholds(trainer, dataset, id2label, threshold_tuning=False, thresholds=None):
    """
    Predicts using trainer.predict(), with optional thresholds tuning.

    Parameters:
      - trainer         : Hugging Face Trainer or CustomTrainer instance
      - dataset         : dataset to predict on
      - id2label        : dictionary mapping label indices (int) to label names (string)
      - threshold_tuning: boolean to enable threshold tuning
      - thresholds      : custom thresholds for classification (optional)

    Returns:
      - predictions                               : final binary predictions
      - true_labels                               : ground truth labels from the dataset
      - best_thresholds (if threshold_tuning=True): optimized threshold for each label
      - best_metrics    (if threshold_tuning=True): dictionary of best F1, precision_for_best_f1 and recall_for_best_f1 for each label
    """
    # Predict
    predictions_output = trainer.predict(dataset)        # <class 'transformers.trainer_utils.PredictionOutput'> len   = 3
    predictions        = predictions_output.predictions  # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    label_ids          = predictions_output.label_ids    # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    metrics            = predictions_output.metrics      # <class 'dict'                                         len   = 10

    print(f"predictions: {type(predictions)} shape={predictions.shape}\n{predictions}")
    print(f"label_ids  : {type(label_ids)}   shape={label_ids.shape}  \n{label_ids}")
    print(f"metrics    : {type(metrics)}     len={len(metrics)}       \n{json.dumps(metrics, indent=4)}")

    logits             = predictions
    true_labels        = label_ids

    # Convert logits to probabilities
    probabilities = 1 / (1 + np.exp(-logits))            # <class 'numpy.ndarray'>                               shape = (12, 6) TODO
    print(f"probabilities: {type(probabilities)} shape={probabilities.shape}")

    best_thresholds, best_metrics = tune_thresholds(true_labels, probabilities, id2label)
    print(f"best_thresholds: {type(best_thresholds)} shape={best_thresholds.shape}\n{best_thresholds}")
    print(f"best_metrics   : {type(best_metrics)}    len={len(best_metrics)}\n{best_metrics}")

    metrics = compute_metrics_with_threshold(probabilities, label_ids, best_thresholds, id2label)

    return metrics

In [ ]:
evaluation_trainer_predict_results = predict_with_thresholds(trainer, validation_dataset, id2label, threshold_tuning=True, thresholds=None)

except_report = {k: v for k, v in evaluation_trainer_predict_results.items() if k!='classification_report'}
report        = evaluation_trainer_predict_results['classification_report']
print(f"evaluation_trainer_predict_results: {type(except_report)} len={len(except_report)}\n{json.dumps(except_report, indent=4)}")
print(f"evaluation_trainer_predict_results['classification_report']: {type(report)} len={len(report)}\n{report}")


In [ ]:
print("trainer.predict successfully completed")

## Evaluation: trainer.predict results: save locally and upload to HF Hub

In [ ]:
evaluation_trainer_predict_results_path = "evaluation_trainer_predict_results.json"
with open(evaluation_trainer_predict_results_path, "w") as f:
  json.dump(evaluation_trainer_predict_results, f)

print(f"evaluation_trainer_predict_results successfully saved locally to {evaluation_trainer_predict_results_path}")

In [ ]:
upload_file(
    path_or_fileobj = evaluation_trainer_predict_results_path,
    path_in_repo    = evaluation_trainer_predict_results_path,
    repo_id         = repo_id,
    repo_type       = 'dataset',
    commit_message  = commit_message
)

print(f"evaluation_trainer_evaluate_results successfully uploaded to HF Hub {repo_id} as {evaluation_trainer_evaluate_results_path}")

In [ ]:
"""
upload_file(
    path_or_fileobj = f"{phase_evaluate}_results.csv",
    path_in_repo    = f"{phase_evaluate}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_evaluate}_metrics.json",
    path_in_repo    = f"{phase_evaluate}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_evaluate.json',
    path_in_repo    = 'trainer_evaluate.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
"""

In [ ]:
raise Exception("I'm here")

# Prediction/Inference step

## Prediction/Inference: trainer.evaluate

## Prediction/Inference: trainer.predict

First test results

In [ ]:
phase_test = 'test_model_eval'

In [ ]:
get_results(
    model      = model,
    dataset    = test_dataset,
    batch_size = batch_size,
    threshold  = threshold,
    phase      = phase_test
)

In [ ]:
print("First test successfully completed.")

Second test results

In [ ]:
trainer_predict = trainer.predict(test_dataset)
print(f"trainer_predict: {type(trainer_predict)} len={len(trainer_predict)}\n{trainer_predict}")
print(f"trainer_predict.predictions: {type(trainer_predict.predictions)} shape={trainer_predict.predictions.shape}")  # Model logits
print(f"trainer_predict.label_ids: {type(trainer_predict.label_ids)} shape={trainer_predict.label_ids.shape}")        # Ground truth labels
print(f"trainer_predict.metrics: {type(trainer_predict.metrics)} len={len(trainer_predict.metrics)}")

In [ ]:
trainer_predict_json_serializable = {
    'predictions': trainer_predict.predictions.tolist(),  # Convert Numpy array to list
    'label_ids'  : trainer_predict.label_ids.tolist(),    # Convert Numpy array to list
    'metrics'    : trainer_predict.metrics                # Dictionary is already serializable
}

In [ ]:
file_path = "trainer_predict.json"
with open(file_path, "w") as f:
  json.dump(trainer_predict_json_serializable, f)
print(f"Test output successfully saved to {file_path}.")

In [ ]:
print("Second test successfully completed.")

Upload Test Results to HF repo_id_dataset

In [ ]:
upload_file(
    path_or_fileobj = f"{phase_test}_results.csv",
    path_in_repo    = f"{phase_test}_results.csv",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = f"{phase_test}_metrics.json",
    path_in_repo    = f"{phase_test}_metrics.json",
    repo_id         = HF_name,
    repo_type       = 'dataset'
)
upload_file(
    path_or_fileobj = 'trainer_predict.json',
    path_in_repo    = 'trainer_predict.json',
    repo_id         = HF_name,
    repo_type       = 'dataset'
)

# Upload the tokenizer and the model to HF Hub

In [ ]:
tokenizer.push_to_hub(repo_id=repo_id, revision=run_name, commit_message=commit_message)
model.push_to_hub(repo_id=repo_id, revision=run_name, commit_message=commit_message)

print(f"tokenizer and model successfully uploaded to HF Hub at {repo_id}")

# Download the tokenizer and the model
It's also a check whether the upload was successful.

In [ ]:
print("Tokenizer")
tokenizer = LongformerTokenizerFast.from_pretrained(repo_id, revision=run_name)
print("Model")
model     = LongformerForSequenceClassification.from_pretrained(repo_id, revision=run_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)

print()
print(f"outputs: {type(outputs)} {outputs.keys()}\n{outputs}")

In [ ]:
print("It's the end")